In [7]:
import sys
sys.path.append("./Preprocessing")
sys.path.append("./Graph")

import pandas as pd
import torch
from feature_engineering import FeatureEngineering
from Resampling import Resampling
from graph_construction import GraphConstruction

In [2]:
dataset = pd.read_csv('C:/Users/ruben/OneDrive/Desktop/Datasets/original_dataset.csv')

# Apply feature engineering on the dataset
fe = FeatureEngineering(dataset)
processed_dataset = fe.apply_feature_engineering()

# Apply resampling on the dataset
resampler = Resampling(processed_dataset, test_size=0.4, random_state=None)
final_dataset = resampler.apply_resampling()

Feature engineering completed.
Fraud rate in training set before resampling: 0.56%
Fraud rate in testing set: 0.60%
Fraud rate in training set after resampling: 50.00%
Fraud rate in testing set after resampling: 0.60%


In [3]:
final_dataset.head()

,index,category,gender,city,state,job,hour,day,month,weekday,age,distance_km,hours_diff_bet_trans,amt_log,is_fraud,cc_user,merchant_num
0,0,0.559000,0,0.364171,0.173886,0.581867,1.000000,1.000000,0.181818,0.166667,0.365854,0.807958,0.000000,0.246234,0,429.0,152.0
1,1,1.000000,1,0.270945,0.594000,0.261154,0.217391,0.833333,0.181818,0.166667,0.951220,0.085104,0.016129,0.375495,0,420.0,113.0
2,2,0.912004,0,0.450446,0.455828,0.469095,0.391304,0.300000,0.454545,0.333333,0.390244,0.640710,0.032258,0.397857,0,204.0,204.0
3,3,0.625724,0,0.359537,0.270716,0.787249,1.000000,0.333333,0.545455,0.500000,0.475610,0.414337,0.000000,0.662805,1,226.0,218.0
4,4,0.835582,1,0.452941,0.308232,0.156365,0.956522,0.066667,0.272727,0.666667,0.682927,0.171983,0.013441,0.653276,1,190.0,393.0


In [5]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [8]:
graph_constructor = GraphConstruction(final_dataset)
data = graph_constructor.apply_graph_construction()

Fraud Percentage in Train Mask: 49.20%
Fraud Percentage in Test Mask: 0.59%
Fraud Percentage in Val Mask: 0.60%
Graph Construction Successful!
